In [85]:
# Pkg.add("DataFrames")
# Pkg.add("Requests")
# Pkg.add("MIToS")
using DataFrames
using Requests
using MIToS.SIFTS

function getCSA()
    url = "http://www.ebi.ac.uk/thornton-srv/databases/CSA/downloads/CSA_2_0_121113.txt"
    csa_file = "csa.csv"
    
    if (stat(csa_file).mode == 0) 
        stream = Requests.get_streaming(url)
        headers(stream)
        response = readall(stream) 
        open(csa_file, "w") do file
            write(file, response)
        end        
    end
    readtable(csa_file)
end

function getUniProtChainResidues(pdb_accessor, the_chain, the_number)
    siftsfile = string(pdb_accessor,".xml.gz")
    if (stat(siftsfile).mode == 0)
        siftsfile = downloadsifts(pdb_accessor)
    end
    siftsresidues = read(siftsfile, SIFTSXML, chain=the_chain, missings=false) # Array{SIFTSResidue,1}

    indexes = findobjects(siftsresidues, dbPDB, SIFTS.In(:number, the_number))

    [ get(siftsresidues[ idx ].UniProt) for idx in indexes ] # UniProt data of the basic residues
end


getUniProtChainResidues (generic function with 1 method)

In [ ]:
catalytic_sites = getCSA()
head(catalytic_sites)

In [ ]:
# obtengo las que tiene evidencia en la literatura
literature_catalytic_sites= catalytic_sites[catalytic_sites[:EVIDENCE_TYPE] .== "LIT",:]
size(literature_catalytic_sites)

In [ ]:
getUniProtChainResidues("12as","A",[46,100,116])

In [ ]:
getUniProtChainResidues("132l","A",[52,35])